# Load Data and inspect

In [1]:
from xgboost import XGBClassifier

/resources/common/.virtualenv/python2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
import pandas as pd
import numpy as np
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import LabelEncoder

In [12]:
tf1 = pd.read_csv('transaction_features1.csv')

In [13]:
ttrain = pd.read_csv('tj_05_training.csv', header=None)
ttest = pd.read_csv('tj_05_test.csv', header=None)

In [14]:
ttrain.columns = ['card_no', 'gender']
ttest.columns = ['card_no']

In [1]:
# Normalize
tf1 = tf1.fillna(0)

scaler = MinMaxScaler().fit(all_txn)
tf1['avg_txn_lg'] = transformer.transform(tf1['avg_txn'].reshape(-1, 1))
tf1['max_txn_lg'] = transformer.transform(tf1['max_txn'].reshape(-1, 1))
tf1['min_txn_lg'] = transformer.transform(tf1['min_txn'].reshape(-1, 1))

tf1['avg_txn_lgscale'] = scaler.transform(tf1['avg_txn_lg'].reshape(-1, 1))
tf1['max_txn_lgscale'] = scaler.transform(tf1['max_txn_lg'].reshape(-1, 1))
tf1['min_txn_lgscale'] = scaler.transform(tf1['min_txn_lg'].reshape(-1, 1))

NameError: name 'tf1' is not defined

In [16]:
ttrain_full = pd.merge(ttrain, tf1, on="card_no")
ttest_full = pd.merge(ttest, tf1, on="card_no")

In [17]:
ttrain_full.head()

,card_no,gender,avg_txn,max_txn,min_txn,no_tran,pay_when_work,has_mer_id
0,1234000000000792,1,1907.6531,25000,50,98,1,1
1,1234000000004032,1,50.0000,50,50,1,0,1
2,1234000000007128,0,50.0000,50,50,1,0,1
3,1234000000024393,1,1111.1111,1500,1000,9,1,1
4,1234000000027939,0,50.0000,50,50,1,0,1


# Data Transformation

In [18]:
from sklearn.cross_validation import cross_val_score

In [21]:
# Feature 3 + Categorical + Remove duplicated predictors
tf2 = pd.read_csv('feature3.csv')
buy_men = pd.read_csv('buy_men.csv')
buy_women = pd.read_csv('buy_women.csv')
buy_cosmetic = pd.read_csv('buy_cosmetic.csv')

In [22]:
tf2.head()

,card_no,avg_txn,max_txn,min_txn,max_cat_code,no_tran
0,1234000000000001,2514.4231,100000,200,8062.0,52
1,1234000000000002,50.0000,50,50,NaN,1
2,1234000000000003,50.0000,50,50,NaN,1
3,1234000000000004,851.9231,4900,50,8999.0,52
4,1234000000000005,178.5714,250,100,5411.0,7


In [23]:
# Log Transformation
from sklearn.preprocessing import MinMaxScaler
from sklearn.cross_validation import cross_val_score
from sklearn.preprocessing import FunctionTransformer

transformer = FunctionTransformer(np.log1p)

tf2['avg_txn_lg'] = transformer.transform(tf2['avg_txn'].reshape(-1, 1))
tf2['max_txn_lg'] = transformer.transform(tf2['max_txn'].reshape(-1, 1))
tf2['min_txn_lg'] = transformer.transform(tf2['min_txn'].reshape(-1, 1))

# Fill NA
tf2 = tf2.fillna(0)

In [24]:
# Scale Number
all_txn = tf2['avg_txn'].append(tf2['max_txn']).append(tf2['min_txn']).reshape(-1,1)
all_txn = transformer.transform(all_txn)

scaler = MinMaxScaler().fit(all_txn)
tf2['avg_txn_lgscale'] = scaler.transform(tf2['avg_txn_lg'].reshape(-1, 1))
tf2['max_txn_lgscale'] = scaler.transform(tf2['max_txn_lg'].reshape(-1, 1))
tf2['min_txn_lgscale'] = scaler.transform(tf2['min_txn_lg'].reshape(-1, 1))

In [25]:
# Label Encode
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder().fit(tf2['max_cat_code'])
tf2['max_cat_encode'] = le.transform(tf2['max_cat_code'])

In [26]:
# Add Categorical Vsriables
tf2_m = pd.merge(tf2, buy_men, on="card_no", how="left")
tf2_mw = pd.merge(tf2_m, buy_women, on="card_no", how="left")
tf2_mwc = pd.merge(tf2_mw, buy_cosmetic, on="card_no", how="left")

# Fill NA
tf2_mwc = tf2_mwc.fillna(0)

In [27]:
tf2_mwc.head()

,card_no,avg_txn,max_txn,min_txn,max_cat_code,no_tran,avg_txn_lg,max_txn_lg,min_txn_lg,avg_txn_lgscale,max_txn_lgscale,min_txn_lgscale,max_cat_encode,buy_men,buy_women,buy_cosmetic
0,1234000000000001,2514.4231,100000,200,8062.0,52,7.830196,11.512935,5.303305,0.303643,0.590491,0.106824,135,0.0,0.0,0.0
1,1234000000000002,50.0000,50,50,0.0,1,3.931826,3.931826,3.931826,0.000000,0.000000,0.000000,0,0.0,0.0,0.0
2,1234000000000003,50.0000,50,50,0.0,1,3.931826,3.931826,3.931826,0.000000,0.000000,0.000000,0,0.0,0.0,0.0
3,1234000000000004,851.9231,4900,50,8999.0,52,6.748669,8.497195,3.931826,0.219403,0.355596,0.000000,152,0.0,1.0,0.0
4,1234000000000005,178.5714,250,100,5411.0,7,5.190573,5.525453,4.615121,0.098044,0.124127,0.053222,19,0.0,0.0,0.0


In [28]:
tf2_mwc['buy_men'] = tf2_mwc['buy_men'].astype(int)
tf2_mwc['buy_women'] = tf2_mwc['buy_women'].astype(int)
tf2_mwc['buy_cosmetic'] = tf2_mwc['buy_cosmetic'].astype(int)

In [29]:
# Merge Card_no
ttrain_full = pd.merge(ttrain, tf2_mwc, on="card_no")
ttest_full = pd.merge(ttest, tf2_mwc, on="card_no")
ttrain_full.head()

,card_no,gender,avg_txn,max_txn,min_txn,max_cat_code,no_tran,avg_txn_lg,max_txn_lg,min_txn_lg,avg_txn_lgscale,max_txn_lgscale,min_txn_lgscale,max_cat_encode,buy_men,buy_women,buy_cosmetic
0,1234000000000792,1,1907.6531,25000,50,8062.0,98,7.554153,10.126671,3.931826,0.282142,0.482515,0.000000,135,0,0,0
1,1234000000004032,1,50.0000,50,50,0.0,1,3.931826,3.931826,3.931826,0.000000,0.000000,0.000000,0,0,0,0
2,1234000000007128,0,50.0000,50,50,0.0,1,3.931826,3.931826,3.931826,0.000000,0.000000,0.000000,0,0,0,0
3,1234000000024393,1,1111.1111,1500,1000,6011.0,9,7.014015,7.313887,6.908755,0.240071,0.263428,0.231872,78,0,0,0
4,1234000000027939,0,50.0000,50,50,0.0,1,3.931826,3.931826,3.931826,0.000000,0.000000,0.000000,0,0,0,0


# Predict

In [69]:
# Settings
use_columns = [
                'avg_txn_lg',
                'max_txn_lg',
                'min_txn_lg',
                'buy_men',
                'buy_women',
                'buy_cosmetic',
]

# from sklearn.naive_bayes import GaussianNB
# model = GaussianNB()

# from sklearn.ensemble import RandomForestClassifier
# model = RandomForestClassifier(n_estimators=10)

from sklearn.ensemble import AdaBoostClassifier
model = AdaBoostClassifier(n_estimators=50, learning_rate=1)

# from sklearn import svm
# model = svm.SVC()

# model = XGBClassifier(max_depth=5)

testsze = 0.3

In [70]:
# Test Model
x_train, x_test, y_train, y_test = train_test_split(
    ttrain_full[use_columns],
    ttrain_full['gender'], test_size = testsze)

estimator = model.fit(x_train, y_train)

np.mean(cross_val_score(estimator, x_test, y_test, cv=10))

0.56401644027738096

In [52]:
# from sklearn.grid_search import GridSearchCV
# parameters = {'learning_rate':[0.05,0.1,0.2,0.3,0.5,0.7,1], 'n_estimators':[30,50,70,100,200]}
# model = AdaBoostClassifier()
# clf = GridSearchCV(model, parameters)
# clf.fit(x_train, y_train)
# print(clf.best_params_)

In [71]:
# Save File
predict_y = estimator.predict(ttest_full[use_columns])
np.savetxt('5.txt', predict_y, fmt="%d")